In [1]:
import random
import networkx as nx
from tqdm import tqdm
from generate_asymp_3_reg_graph import create_random_3_reg_graph
from matplotlib import pyplot as plt

def get_smallest_cycle_larger_3_no_cycles_in_subgraph(G, useThreshold = False):
    n = len(G.nodes())
    length_of_smallest_cycle = float('inf')
    vertex_of_smallest_cycle = None

    progress_bar = tqdm(total=n, leave=False)

    for node in G.nodes():
        progress_bar.update(1)
        for neighbor in G.neighbors(node):
            if abs(neighbor - node) == 1 or (node == 0 and neighbor == n - 1) or (node == n - 1 and neighbor == 0):
                continue
            else:
                length = min(abs(n - neighbor + node), abs(neighbor - node)) + 1
                if (length < length_of_smallest_cycle and not useThreshold) or (length < length_of_smallest_cycle and useThreshold and length < 10):
                    if length == 3:
                        continue
                    else:
                        if abs(n - neighbor + node) < abs(neighbor - node):
                            helper_subgraph = G.subgraph(list(range(max(node, neighbor) + 1, n)) + list(range(0, min(node, neighbor))))
                        else:
                            helper_subgraph = G.subgraph(range(min(node, neighbor) + 1, max(node, neighbor)))
                    try:
                        cycle = nx.find_cycle(helper_subgraph)
                        if cycle:  # If a cycle is found, continue
                            continue
                    except nx.exception.NetworkXNoCycle:
                        pass
                    length_of_smallest_cycle = length
                    if abs(n - neighbor + node) < abs(neighbor - node):
                        vertex_of_smallest_cycle = max(node, neighbor)
                    else:
                        vertex_of_smallest_cycle = min(node, neighbor)
                break
        if length_of_smallest_cycle == 4:
            break
    return length_of_smallest_cycle, vertex_of_smallest_cycle


def alorithm1(H, length_of_smallest_cycle):
    n = len(H.nodes())
    colors = set([0, 1, 2])

    sudoku_colors = {node: -1 for node in H.nodes()}

    sudoku_colors[length_of_smallest_cycle - 2] = 0
    for i in range(length_of_smallest_cycle-3):
        sudoku_colors[i] = i % 3
    sudoku_colors[length_of_smallest_cycle - 3] = random.choice(list(colors - set([sudoku_colors[length_of_smallest_cycle - 4], sudoku_colors[length_of_smallest_cycle - 2]])))

    all_colors = sudoku_colors.copy()

    pointer = length_of_smallest_cycle - 2
    for i in tqdm(range(length_of_smallest_cycle - 1, n-1)):
        if pointer == i-1:
            neighbor = max(list(H[i].keys()), key=lambda x: abs(x - i))
            if neighbor < i and len(colors - set([all_colors[neighbor], all_colors[i-1]])) == 1:
                all_colors[i] = list(colors - set([all_colors[neighbor], all_colors[i-1]]))[0]
                pointer = i
            elif all_colors[neighbor] == all_colors[i-1] or neighbor > i:
                all_colors[i] = random.choice(list(colors - set([all_colors[i-1]])))
        elif pointer < i-1:
            neighbor = max(list(H[i].keys()), key=lambda x: abs(x - i))
            previous_neighbor = max(list(H[i-1].keys()), key=lambda x: abs(x - (i-1)))
            if neighbor < i:
                all_colors[i] = random.choice(list(colors - set([all_colors[neighbor], all_colors[i-1]])))
                previous_pointer = pointer
                if (previous_neighbor > i-1 and all_colors[i-2] == all_colors[i]) or (previous_neighbor < i-1 and all_colors[previous_neighbor] == all_colors[i]):
                    sudoku_colors[i-1] = all_colors[i-1]
                    if all_colors[neighbor] == all_colors[i-1]:
                        pointer = i-1
                    else:
                        pointer = i
                if neighbor < previous_pointer:
                    sudoku_colors[i] = all_colors[i]
                    pointer = i
            elif neighbor > i:
                if previous_neighbor < i-1:
                    all_colors[i] = random.choice(list(colors - set([all_colors[previous_neighbor], all_colors[i-1]])))
                elif previous_neighbor > i-1:
                    all_colors[i] = random.choice(list(colors - set([all_colors[i-2], all_colors[i-1]])))
                sudoku_colors[i] = all_colors[i]
                pointer = i
    all_colors[n-1] = random.choice(list(colors - set([all_colors[n-2], all_colors[0]])))
    sudoku_colors[n-1] = all_colors[n-1]
    return all_colors, sudoku_colors


def algorithm2(H, length_of_smallest_cycle):
    n = len(H.nodes())
    colors = set([0, 1, 2])

    sudoku_colors = {node: -1 for node in H.nodes()}

    sudoku_colors[length_of_smallest_cycle - 2] = 0
    for i in range(length_of_smallest_cycle-3):
        sudoku_colors[i] = i % 3
    sudoku_colors[length_of_smallest_cycle - 3] = random.choice(list(colors - set([sudoku_colors[length_of_smallest_cycle - 4], sudoku_colors[length_of_smallest_cycle - 2]])))

    all_colors = sudoku_colors.copy()

    pointer = length_of_smallest_cycle - 2
    for i in tqdm(range(length_of_smallest_cycle - 1, n-1)):
        if pointer == i-1:
            neighbor = max(list(H[i].keys()), key=lambda x: abs(x - i))
            if neighbor < i and all_colors[neighbor] != all_colors[i-1]:
                all_colors[i] = list(colors - set([all_colors[neighbor], all_colors[i-1]]))[0]
                pointer = i
            elif all_colors[neighbor] == all_colors[i-1] or neighbor > i:
                all_colors[i] = random.choice(list(colors - set([all_colors[i-1]])))
        elif pointer < i-1:
            neighbor = max(list(H[i].keys()), key=lambda x: abs(x - i))
            previous_neighbor = max(list(H[i-1].keys()), key=lambda x: abs(x - (i-1)))
            if neighbor < i:
                if all_colors[neighbor] == all_colors[i-1]:
                    all_colors[i] = random.choice(list(colors - set([all_colors[i-2], all_colors[neighbor]])))  
                else:  
                    all_colors[i] = random.choice(list(colors - set([all_colors[neighbor], all_colors[i-1]])))
                previous_pointer = pointer
                if (previous_neighbor > i-1 and all_colors[i-2] == all_colors[i]) or (previous_neighbor < i-1 and all_colors[previous_neighbor] == all_colors[i]):
                    sudoku_colors[i-1] = all_colors[i-1]
                    if all_colors[neighbor] == all_colors[i-1]:
                        pointer = i-1
                    else:
                        pointer = i
                if neighbor < previous_pointer:
                    sudoku_colors[i] = all_colors[i]
                    pointer = i
            elif neighbor > i:
                if previous_neighbor < i-1:
                    all_colors[i] = random.choice(list(colors - set([all_colors[previous_neighbor], all_colors[i-1]])))
                elif previous_neighbor > i-1:
                    all_colors[i] = random.choice(list(colors - set([all_colors[i-2], all_colors[i-1]])))
                sudoku_colors[i] = all_colors[i]
                pointer = i
    all_colors[n-1] = random.choice(list(colors - set([all_colors[n-2], all_colors[0]])))
    sudoku_colors[n-1] = all_colors[n-1]
    return all_colors, sudoku_colors

In [ ]:
n = 1000000

#random.seed(0)

# Generate a random 3-regular graph with n vertices by assuming that there is a hamilton cycle 1,2,3,...,n and randomly matching the nodes afterwards
G = create_random_3_reg_graph(n)
print(G)

length_of_smallest_cycle, vertex_of_smallest_cycle = get_smallest_cycle_larger_3_no_cycles_in_subgraph(G, useThreshold=True)
H = nx.relabel_nodes(G, lambda x: (x + n - vertex_of_smallest_cycle - 1) % n)

print((length_of_smallest_cycle, vertex_of_smallest_cycle))
print(get_smallest_cycle_larger_3_no_cycles_in_subgraph(H))

all_colors, sudoku_colors = alorithm1(H, length_of_smallest_cycle)
node_colors = []
for node in H.nodes():
    if all_colors[node] == -1:
        node_colors.append('#1f78b4')
    else:
        if all_colors[node] == 0:
            node_colors.append('red')
        elif all_colors[node] == 1:
            node_colors.append('green')
        else:
            node_colors.append('yellow')
        #node_colors.append(best_coloring[node])

candidate_sudoku_colors = []
for node in H.nodes():
    if sudoku_colors[node] == -1:
        candidate_sudoku_colors.append('#1f78b4')
    else:
        #sudoku_colors.append(best_sudoku_coloring[node])
        if sudoku_colors[node] == 0:
            candidate_sudoku_colors.append('red')
        elif sudoku_colors[node] == 1:
            candidate_sudoku_colors.append('green')
        else:
            candidate_sudoku_colors.append('yellow')

number_of_vertices_in_sudoku_set_algo1 = sum(1 for key in sudoku_colors.keys() if sudoku_colors[key] >= 0)

sudoku_colors_algo2 = algorithm2(H, length_of_smallest_cycle)[1]
number_of_vertices_in_sudoku_set_algo2 = sum(1 for key in sudoku_colors_algo2.keys() if sudoku_colors_algo2[key] >= 0)

print("Ratio of Sudoku set Algo1: ", number_of_vertices_in_sudoku_set_algo1/n)
print("Ratio of Sudoku set Algo2: ", number_of_vertices_in_sudoku_set_algo2/n)

if n<=100:
    posH = nx.kamada_kawai_layout(H)
    nx.draw(H, posH, with_labels=True, node_color=candidate_sudoku_colors)
    plt.show()

    nx.draw(H, posH, with_labels=True, node_color=node_colors)
    plt.show()

Graph with 500000 nodes and 750000 edges


(4, 425149)


(4, 499999)


100%|██████████| 499996/499996 [00:02<00:00, 214034.34it/s]

Ratio of Sudoku set Algo1:  0.479124
Ratio of Sudoku set Algo2:  0.450054
